# Neuvisys notebook

Display multiple information from a spiking neural network experiment.

In [76]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import pprint
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pdf2image import convert_from_path
from src.spiking_network.network.neuvisys import SpikingNetwork
from src.spiking_network.analysis.network_display import (
    display_network,
    load_array_param,
    complex_cells_directions,
    complex_cell_disparities,
)
from src.spiking_network.analysis.rl_analysis import (
    value_plot,
    policy_plot)
from src.spiking_network.analysis.network_statistics import (
    event_vs_network_activity,
    rf_matching,
    direction_norm_length,
    orientation_norm_length,
    direction_selectivity,
    orientation_selectivity,
    update_dataframe,
    disparity_histogram,
    inhibition_weight_against_orientation,
    inhibition_weight_against_disparity,
    lateral_inhibition_weight_sum,
)
from src.spiking_network.gabor.gabor_fit import gabor_fit_one_patch
from src.spiking_network.analysis.spike_train import (
    spike_trains,
    raster_plot,
    event_plot,
    time_histogram,
    fast_time_histogram,
    spike_rate_histogram,
    isi_histogram,
    instantaneous_rates,
    correlation_coeficient_matrix,
    activity_comparison,
)
from src.spiking_network.gabor.gabbor_fitting import (
    create_gabor_basis,
    hists_preferred_orientations,
    plot_preferred_orientations,
)
from src.spiking_network.analysis.parameter_analysis import (
    correlation_matrix,
    scatter_mat,
)

os.chdir("/home/thomas/neuvisys-analysis/src")
home = "/home/thomas/"

SMALL_SIZE = 20
MEDIUM_SIZE = 25
BIGGER_SIZE = 30

plt.rc('figure', figsize=(30, 8))
plt.rc('font', size=SMALL_SIZE)  # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE,
       labelsize=MEDIUM_SIZE)  # fontsize of the axes title, fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)  # legend fontsize

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Network

loads various information of the spiking neural network into a SpikingNetwork instance.

In [6]:
# network_path = "/home/thomas/Networks/handcraft/disparity/inhibition_learning/+2/validation/"
network_path = "/home/thomas/Networks/simulation/rl/orientation_task/3actions/network_jitter_3000/"
spinet = SpikingNetwork(network_path, loading=True)

## Weights analysis

Display of the learned receptive fields of the different neuronal layers

In [ ]:
display_network([spinet])

print("Simple cells weights")
if spinet.conf["nbCameras"] == 2:
    display(convert_from_path(network_path + "figures/0/weight_sharing_combined.pdf")[0])
else:
    display(convert_from_path(network_path + "figures/0/weight_sharing_0.pdf")[0])

print("Complex cells weights (average over all simple cells)")
for i in range(spinet.l_shape[1, 2]):
    display(convert_from_path(network_path + "figures/1/complex_weights_depth_" + str(i) + ".pdf")[0])
    break

In [ ]:
network_path = "/home/thomas/Desktop/Experiment/network_0/"
spinet = SpikingNetwork(network_path)
ws = []
cs = []
for i in range(1000):
    weights = []
    confs = []
    if os.path.exists(network_path + "weights/intermediate_" + str(i) + "/0/0.npy"):
        for j in range(spinet.l_shape[0][2]):
            w = np.load(network_path + "weights/intermediate_" + str(i) + "/0/" + str(j) + ".npy")
            # with open(network_path + "weights/intermediate_" + str(i) + "/0/" + str(j) + ".json") as file:
            #     confs.append(json.load(file)["count_spike"])
            weights.append(w)
        ws.append(weights)
        cs.append(confs)
    else:
        break

    # spinet.weights[0] = np.array(weights)
    # display_network([spinet])
    # shutil.copy(network_path + "figures/0/weight_sharing_combined.pdf", "/home/thomas/Bureau/weights/" + str(i) + ".pdf")

    # disparities, residuals = rf_matching(spinet)
    # plt.figure()
    # plt.hist(disparities[:, 0], bins=np.arange(-8, 9), align="left")
    # plt.title("Histogram of simple cell disparities")
    # plt.xlabel("Disparity (px)")
    # plt.ylabel("Count")
    # plt.savefig("/home/thomas/Bureau/disparities/" + str(i), bbox_inches="tight")

ws = np.array(ws)
cs = np.array(cs)

In [ ]:
total_sum = np.sum(ws, axis=tuple(np.arange(1, ws.ndim)))
weight_diff = np.abs(np.diff(ws, axis=0))
sum_diff = np.sum(weight_diff, axis=tuple(np.arange(1, weight_diff.ndim)))

plt.figure()
plt.title("Weight change percentage over time")
plt.ylabel("Weight change in %")
plt.xlabel("Time (s)")
plt.plot(100 * sum_diff / total_sum[:-1])

In [ ]:
inhibition_weight_against_orientation(spinet)
# inhibition_weight_against_disparity(spinet)
lateral_inhibition_weight_sum(spinet)

## Direction and Orientation selectivity

In [ ]:
rotations = np.array([0, 23, 45, 68, 90, 113, 135, 158, 180, 203, 225, 248, 270, 293, 315, 338])
dir_vec, ori_vec = complex_cells_directions(spinet, rotations)

angles = np.pi * rotations / 180

dirs = []
dis = []
for i in range(len(spinet.neurons[1])):
    dirs.append(direction_norm_length(spinet.directions[:, i], angles))
    dis.append(direction_selectivity(spinet.directions[:, i]))
oris = []
ois = []
for i in range(len(spinet.neurons[1])):
    oris.append(orientation_norm_length(spinet.orientations[:, i], angles[0:8]))
    ois.append(orientation_selectivity(spinet.orientations[:, i]))

## Disparity selectivity

In [ ]:
plt.imshow(spinet.disparities)
plt.colorbar(orientation='horizontal')

plt.figure()
mean = np.mean(spinet.disparities, axis=1)
plt.plot(mean)

In [ ]:
disparities = np.array([-8, -7, -6, -5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 7, 8])

dir_vec, ori_vec = complex_cell_disparities(spinet, spinet.disparities, disparities)

# angles = np.pi * rotations / 180
#
# dirs = []
# dis = []
# for i in range(len(spinet.neurons[1])):
#     dirs.append(direction_norm_length(spinet.directions[:, i], angles))
#     dis.append(direction_selectivity(spinet.directions[:, i]))
# oris = []
# ois = []
# for i in range(len(spinet.neurons[1])):
#     oris.append(orientation_norm_length(spinet.orientations[:, i], angles[0:8]))
#     ois.append(orientation_selectivity(spinet.orientations[:, i]))

In [ ]:
event_vs_network_activity(spinet, "/home/thomas/Videos/natural/pavin_aquisitions/pavin.h5", bins=50)

## Spike analysis

Displaying information related to spike timings in the network.

This includes:
- Eventplots
- Time histograms
- Inter-spike histograms
- Raster plots
- Instantaneous rates
- Correlation coefficient matrices
- spike rate histograms

In [ ]:
full = False
for layer, layer_spike_train in enumerate(spinet.spikes):
    subset = slice(0, len(layer_spike_train))
    print(subset)
    print("Layer " + str(layer + 1) + ", nb neurons = " + str(layer_spike_train[subset].shape[0]) + ":")

    if full:
        sts = spike_trains(layer_spike_train[subset])
        path = network_path + "figures/"
        # raster_plot(sts, layer, path)
        event_plot(sts, layer, path)
        time_histogram(sts, layer, 50, path)
        spike_rate_histogram(layer_spike_train, layer, path)
        # isi_histogram(sts, layer, path)
        instantaneous_rates(sts, layer, path)
        # correlation_coeficient_matrix(sts, layer, path)
    else:
        fast_time_histogram(layer_spike_train, bins=50, display=True)

#### Compare activity between 2 networks

In [ ]:
spinet1 = SpikingNetwork("/home/thomas/Networks/handcraft/disparity/inhibition_learning/basis_disparity/", loading=True)
spinet2 = SpikingNetwork("/home/thomas/Networks/handcraft/disparity/inhibition_learning/0/validation/", loading=True)
activity_comparison(spinet1, spinet2)

## Compute receptive field disparities

In [ ]:
disparities, residuals = rf_matching(spinet)
disparity_histogram(spinet, disparities)

monocular_count = np.count_nonzero(np.isnan(disparities[:, 0]))
print("There is : ", monocular_count, "monocular cells (left right ratio imbalance of more than 90 %), which is ", 100 * monocular_count / disparities.shape[0], "%")

## Network training information

In [ ]:
value_plot(spinet)

## Policy plot

In [ ]:
policy_plot(spinet)

## Critic and Actor neurons weights

In [ ]:
w = np.zeros((spinet.neurons[2][0].weights.shape[0], spinet.neurons[2][0].weights.shape[1]))
for neuron in spinet.neurons[2]:
    w += np.sum(neuron.weights, axis=2)
plt.figure(figsize=(18, 8))
plt.title("Critic neurons summed weights")
plt.imshow(w.T)
plt.colorbar(orientation="horizontal")
plt.show()

for i, neuron in enumerate(spinet.neurons[3]):
    plt.figure(figsize=(18, 8))
    if i == 0:
        plt.title("Action: camera to the left")
    else:
        plt.title("Action: camera to the right")
    plt.imshow(np.sum(neuron.weights, axis=2).T)
    plt.colorbar(orientation="horizontal")
    plt.show()

## Event rates

In [ ]:
nb_events = np.array(spinet.state["learning_data"]["nbEvents"])[:-1]
t = np.linspace(0, np.max(spinet.spikes[0]), nb_events.size) * 1e-6
plt.figure(figsize=(40, 8))
plt.title("Number of events (every 10ms)")
plt.xlabel("time (s)")
plt.plot(t, nb_events)
plt.show()

## Network configuration parameters

In [ ]:
print("Network config:\n")
pprint.pprint(spinet.conf)
for layer in range(len(spinet.neurons)):
    print("\nLayer " + str(layer) + ":\n")
    pprint.pprint(spinet.neurons[layer][0].conf)

## Load various neuron informations

In [78]:
simpa_decay, compa_decay = load_array_param(spinet, "learning_decay")
simpa_spike, compa_spike = load_array_param(spinet, "count_spike")

# Plot correlations

df = pd.read_csv(home + "Desktop/test/df")
temp = df.drop(columns=["TRACKING", "SYNAPSE_DELAY", "STDP_LEARNING", "MIN_THRESH"])

correlation_matrix(temp.iloc[:, 25:])
scatter_mat(temp.iloc[:, 25:])

AttributeError: 'SpikingNetwork' object has no attribute 'nb_simple_cells'

## Load and create gabor basis

In [ ]:
basis = spinet.generate_weight_mat()
res = gabor_fit_one_patch(basis, rf_size=(10, 10), nb_cameras=2)

In [ ]:
spinet.generate_weight_images()
gabor_params_l = create_gabor_basis(spinet, res, 0, nb_ticks=8, error_thresh=220)

#### Create plots for preferred orientations and directions

In [ ]:
oris, oris_r = hists_preferred_orientations(spinet)
plot_preferred_orientations(spinet, oris, oris_r)